DS GA 1007 Final Project

In [2]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np 

Join the datasets across years and then merge on player name and year

In [ ]:
# Join the bat tracking datasets across years
bat_df_2025 = pd.read_csv('bat_tracking_2025.csv')
bat_df_2025['year'] = 2025
bat_df_2024 = pd.read_csv('bat_tracking_2024.csv')
bat_df_2024['year'] = 2024
bat_df_2023 = pd.read_csv('bat_tracking_2023.csv')
bat_df_2023['year'] = 2023
bat_df_full = pd.concat([bat_df_2025, bat_df_2024, bat_df_2023], ignore_index=True)
bat_df_full

,id,name,swings_competitive,percent_swings_competitive,contact,avg_bat_speed,hard_swing_rate,squared_up_per_bat_contact,squared_up_per_swing,blast_per_bat_contact,blast_per_swing,swing_length,swords,batter_run_value,whiffs,whiff_per_swing,batted_ball_events,batted_ball_event_per_swing,year
0,519317,"Stanton, Giancarlo",448,0.906883,283,80.620201,0.973214,0.300353,0.189732,0.261484,0.165179,8.788955,4,9.010466,165,0.368304,141,0.314732,2025
1,665833,"Cruz, Oneil",822,0.906284,556,78.806448,0.784672,0.321942,0.217762,0.248201,0.167883,7.761988,19,-21.768531,266,0.323601,273,0.332117,2025
2,691406,"Caminero, Junior",1102,0.906996,851,78.584841,0.813067,0.289072,0.223230,0.222092,0.171506,8.420028,21,1.971819,251,0.227768,432,0.392015,2025
3,656180,"Adams, Riley",500,0.904159,341,78.304560,0.792000,0.234604,0.160000,0.181818,0.124000,7.851963,14,-22.656646,159,0.318000,148,0.296000,2025
4,691023,"Walker, Jordan",686,0.905013,465,78.069258,0.778426,0.266667,0.180758,0.200000,0.135569,8.258906,21,-19.146943,221,0.322157,231,0.336735,2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1887,669364,"Edwards, Xavier",97,0.898148,76,64.026054,0.010309,0.342105,0.268041,0.026316,0.020619,6.211680,4,-6.257673,21,0.216495,37,0.381443,2023
1888,650333,"Arraez, Luis",407,0.896476,374,63.837034,0.002457,0.491979,0.452088,0.072193,0.066339,5.866538,3,-7.855588,33,0.081081,210,0.515971,2023
1889,664770,"Lukes, Nathan",13,0.928571,13,63.502551,0.000000,0.076923,0.076923,0.000000,0.000000,6.011915,1,-0.437373,0,0.000000,3,0.230769,2023
1890,664670,"Lopez, Alejo",2,1.000000,2,61.909735,0.000000,0.500000,0.500000,0.000000,0.000000,5.000840,0,0.782719,0,0.000000,1,0.500000,2023


In [10]:
# Join the sprint speed datasets across years
sprint_df_2025 = pd.read_csv('sprint_speed_2025.csv')
sprint_df_2025['year'] = 2025
sprint_df_2024 = pd.read_csv('sprint_speed_2024.csv')
sprint_df_2024['year'] = 2024
sprint_df_2023 = pd.read_csv('sprint_speed_2023.csv')
sprint_df_2023['year'] = 2023
sprint_df_full = pd.concat([sprint_df_2025, sprint_df_2024, sprint_df_2023], ignore_index=True)
sprint_df_full.rename(columns={'last_name, first_name': 'name'}, inplace=True)
sprint_df_full

,name,player_id,team_id,team,position,age,competitive_runs,bolts,hp_to_1b,sprint_speed,year
0,"Turner, Trea",607208,143,PHI,SS,32,265,117.0,4.22,30.3,2025
1,"Scott II, Victor",687363,138,STL,CF,24,141,87.0,4.13,30.2,2025
2,"Witt Jr., Bobby",677951,118,KC,SS,25,245,101.0,4.15,30.2,2025
3,"Buxton, Byron",621439,142,MIN,CF,31,165,56.0,4.13,30.2,2025
4,"Hill, Derek",656537,145,CWS,CF,29,50,24.0,4.21,30.1,2025
...,...,...,...,...,...,...,...,...,...,...,...
1832,"Cabrera, Miguel",408234,116,DET,DH,40,127,NaN,5.09,23.1,2023
1833,"Aguilar, Jesús",542583,133,OAK,1B,33,27,NaN,NaN,23.0,2023
1834,"Grandal, Yasmani",518735,145,CWS,C,34,129,NaN,5.14,22.8,2023
1835,"León, Sandy",506702,140,TEX,C,34,11,NaN,NaN,22.7,2023


In [14]:
# Merge the datasets on player name and year
merged_df = pd.merge(bat_df_full, sprint_df_full, on=['name', 'year'], how='inner')
merged_df

,id,name,swings_competitive,percent_swings_competitive,contact,avg_bat_speed,hard_swing_rate,squared_up_per_bat_contact,squared_up_per_swing,blast_per_bat_contact,...,year,player_id,team_id,team,position,age,competitive_runs,bolts,hp_to_1b,sprint_speed
0,519317,"Stanton, Giancarlo",448,0.906883,283,80.620201,0.973214,0.300353,0.189732,0.261484,...,2025,519317,147,NYY,DH,35,62,NaN,5.22,23.5
1,665833,"Cruz, Oneil",822,0.906284,556,78.806448,0.784672,0.321942,0.217762,0.248201,...,2025,665833,134,PIT,CF,26,164,42.0,4.26,29.2
2,691406,"Caminero, Junior",1102,0.906996,851,78.584841,0.813067,0.289072,0.223230,0.222092,...,2025,691406,139,TB,3B,21,252,NaN,4.62,26.7
3,656180,"Adams, Riley",500,0.904159,341,78.304560,0.792000,0.234604,0.160000,0.181818,...,2025,656180,120,WSH,C,29,79,NaN,4.58,26.6
4,691023,"Walker, Jordan",686,0.905013,465,78.069258,0.778426,0.266667,0.180758,0.200000,...,2025,691023,138,STL,RF,23,136,2.0,4.40,28.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1774,683021,"Stefanic, Michael",73,0.901235,69,64.524445,0.013699,0.507246,0.479452,0.043478,...,2023,683021,108,LAA,2B,27,31,NaN,4.62,26.0
1775,680757,"Kwan, Steven",448,0.899598,408,64.083518,0.002232,0.450980,0.410714,0.046569,...,2023,680757,114,CLE,LF,25,334,3.0,4.22,28.1
1776,669364,"Edwards, Xavier",97,0.898148,76,64.026054,0.010309,0.342105,0.268041,0.026316,...,2023,669364,146,MIA,2B,23,37,3.0,4.18,28.2
1777,650333,"Arraez, Luis",407,0.896476,374,63.837034,0.002457,0.491979,0.452088,0.072193,...,2023,650333,146,MIA,2B,26,269,NaN,4.57,26.1
